# gpt2Medium_text_to_sql

code link: https://huggingface.co/rakeshkiriyath/gpt2Medium_text_to_sql

dataset: https://huggingface.co/datasets/b-mc2/sql-create-context


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

finetunedGPT = GPT2LMHeadModel.from_pretrained("rakeshkiriyath/gpt2Medium_text_to_sql").to('cuda')
finetunedTokenizer = GPT2Tokenizer.from_pretrained("rakeshkiriyath/gpt2Medium_text_to_sql")

def generate_text_to_sql(query, model, tokenizer, max_length=256):
    prompt = f"Translate the following English question to SQL: {query}"

    input_tensor = tokenizer.encode(prompt, return_tensors='pt').to('cuda')

    output = model.generate(input_tensor, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Return only the SQL part (removing the input text)
    sql_output = decoded_output[len(prompt):].strip()

    return sql_output

queryList = ["I need a list of employees who joined the company in the period of last 6 months with a salary hike of 30% ",
             "Give me loginid,status,company of a user who is mapped to the organization XYZ "
             "Find the number heads of the departments who are older than 56."]

for idx, query in enumerate(queryList):

  sql_result = generate_text_to_sql(query, finetunedGPT, finetunedTokenizer)
  print(idx + 1, sql_result,"\n")

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/525 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

1 SELECT employees FROM employees WHERE last_6_months = "6 months" AND salary_hike = "30%" 

2 SELECT COUNT(DISTINCT T1.department_head) FROM employee AS T1 JOIN department AS T2 ON T1.department_id = T2.department_id GROUP BY T1.department_head ORDER BY COUNT(DISTINCT T2.department_name) DESC LIMIT 56 



# t5-small-awesome-text-to-sql

code link: https://huggingface.co/cssupport/t5-small-awesome-text-to-sql

Dataset: https://huggingface.co/datasets/b-mc2/sql-create-context

In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize the tokenizer from Hugging Face Transformers library
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained('cssupport/t5-small-awesome-text-to-sql')
model = model.to(device)
model.eval()

def generate_sql(input_prompt):
    # Tokenize the input prompt
    inputs = tokenizer(input_prompt, padding=True, truncation=True, return_tensors="pt").to(device)

    # Forward pass
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)

    # Decode the output IDs to a string (SQL query in this case)
    generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_sql

# Test the function
#input_prompt = "tables:\n" + "CREATE TABLE Catalogs (date_of_latest_revision VARCHAR)" + "\n" +"query for: Find the dates on which more than one revisions were made."
#input_prompt = "tables:\n" + "CREATE TABLE table_22767 ( \"Year\" real, \"World\" real, \"Asia\" text, \"Africa\" text, \"Europe\" text, \"Latin America/Caribbean\" text, \"Northern America\" text, \"Oceania\" text )" + "\n" +"query for:what will the population of Asia be when Latin America/Caribbean is 783 (7.5%)?."
# input_prompt = "tables:\n" + "CREATE TABLE procedures ( subject_id text, hadm_id text, icd9_code text, short_title text, long_title text ) CREATE TABLE diagnoses ( subject_id text, hadm_id text, icd9_code text, short_title text, long_title text ) CREATE TABLE lab ( subject_id text, hadm_id text, itemid text, charttime text, flag text, value_unit text, label text, fluid text ) CREATE TABLE demographic ( subject_id text, hadm_id text, name text, marital_status text, age text, dob text, gender text, language text, religion text, admission_type text, days_stay text, insurance text, ethnicity text, expire_flag text, admission_location text, discharge_location text, diagnosis text, dod text, dob_year text, dod_year text, admittime text, dischtime text, admityear text ) CREATE TABLE prescriptions ( subject_id text, hadm_id text, icustay_id text, drug_type text, drug text, formulary_drug_cd text, route text, drug_dose text )" + "\n" +"query for:" + "what is the total number of patients who were diagnosed with icd9 code 2254?"
# input_prompt = "tables:\n" + "CREATE TABLE student_course_attendance (student_id VARCHAR); CREATE TABLE students (student_id VARCHAR)" + "\n" + "query for:" + "List the id of students who never attends courses?"
input_prompt = "tables:\n" + "CREATE TABLE head (age INTEGER)" + "\n" + "query for:" + "Find the number heads of the departments who are older than 56."

generated_sql = generate_sql(input_prompt)

print(f"The generated SQL query is: {generated_sql}")
#OUTPUT: The generated SQL query is: SELECT student_id FROM students WHERE NOT student_id IN (SELECT student_id FROM student_course_attendance)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The generated SQL query is: SELECT COUNT(*) FROM head WHERE age > 56


# Mistral-7B-SQL

code link: https://huggingface.co/machinists/Mistral-7B-SQL

dataset: https://huggingface.co/datasets/b-mc2/sql-create-context


In [ ]:
!pip install accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "machinists/Mistral-7B-SQL"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

table_schema = "CREATE TABLE head (age INTEGER)"

question = "How many heads of the departments are older than 56 ?"

system_msg = f" Generate a correct SQL query from the following database schema. \n {table_schema} "

prompt = f"<s>[INST] {system_msg} \n{question} [/INST]"

sequences = pipeline(
    prompt,
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: <s>[INST]  Generate a correct SQL query from the following database schema. 
 CREATE TABLE head (age INTEGER)  
How many heads of the departments are older than 56 ? [/INST] SELECT COUNT(*) FROM head WHERE age > 56


# CodeLlama-7b-Instruct-SQL

Code link: https://huggingface.co/machinists/CodeLlama-7b-Instruct-SQL

Dataset: https://huggingface.co/datasets/b-mc2/sql-create-context

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "machinists/CodeLlama-7b-Instruct-SQL"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

table_schema = "CREATE TABLE head (age INTEGER)"

question = "How many heads of the departments are older than 56 ?"

system_msg = f"<<SYS>> Generate a correct SQL query from the following database schema. \n {table_schema} <</SYS>>"

prompt = f"<s>[INST] {system_msg} \n \n {question} [/INST]"

sequences = pipeline(
    prompt,
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: <s>[INST] <<SYS>> Generate a correct SQL query from the following database schema. 
 CREATE TABLE head (age INTEGER) <</SYS>> 
 
 How many heads of the departments are older than 56 ? [/INST] SELECT COUNT(age) FROM head WHERE age > 56


# CodeLlama-13b-SQL

Code link: https://huggingface.co/machinists/CodeLlama-13b-SQL

dataset: https://huggingface.co/datasets/b-mc2/sql-create-context

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "machinists/CodeLlama-13b-SQL"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

table_schema = "CREATE TABLE head (age INTEGER)"

question = "How many heads of the departments are older than 56 ?"

system_msg = f"<<SYS>> Generate a correct SQL query from the following database schema. \n {table_schema} <</SYS>>"

prompt = f"<s>[INST] {system_msg} \n \n {question} [/INST]"

sequences = pipeline(
    prompt,
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

# toy-sql-28M

code link: https://huggingface.co/Artifact-io/toy-sql-28M

dataset: https://huggingface.co/datasets/b-mc2/sql-create-context

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained("Artifact-io/toy-sql-28M").to(device)
tokenizer = AutoTokenizer.from_pretrained("Artifact-io/toy-sql-28M")

inputs = tokenizer([
"""CREATE TABLE head (age INTEGER)
How many heads of the departments are older than 56?
"""
  ],
  return_tensors="pt",
).to(device)

outputs = model.generate(**inputs, max_new_tokens=200, do_sample=True, top_k=50, top_p=0.95)
text = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0].split("---")[0]
print(text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



SELECT MAX(Xus) FROM head



# T5-LM-Large-text2sql-spider

code link: https://huggingface.co/gaussalgo/T5-LM-Large-text2sql-spider

dataset: https://huggingface.co/datasets/spider


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_path = 'gaussalgo/T5-LM-Large-text2sql-spider'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# question = "What is the average, minimum, and maximum age for all French musicians?"

# schema = """
#    "stadium" "Stadium_ID" int , "Location" text , "Name" text , "Capacity" int , "Highest" int , "Lowest" int , "Average" int , foreign_key:  primary key: "Stadium_ID" [SEP] "singer" "Singer_ID" int , "Name" text , "Country" text , "Song_Name" text , "Song_release_year" text , "Age" int , "Is_male" bool , foreign_key:  primary key: "Singer_ID" [SEP] "concert" "concert_ID" int , "concert_Name" text , "Theme" text , "Year" text , foreign_key: "Stadium_ID" text from "stadium" "Stadium_ID" , primary key: "concert_ID" [SEP] "singer_in_concert"  foreign_key: "concert_ID" int from "concert" "concert_ID" , "Singer_ID" text from "singer" "Singer_ID" , primary key: "concert_ID" "Singer_ID"
# """

question = "How many heads of the departments are older than 56?"
schema = """
    "head" "age" int
"""

input_text = " ".join(["Question: ",question, "Schema:", schema])

model_inputs = tokenizer(input_text, return_tensors="pt")

# Generate outputs
outputs = model.generate(**model_inputs, max_length=512)

# Check and flatten the outputs if necessary
if isinstance(outputs, torch.Tensor):
    outputs = outputs.tolist()

if any(isinstance(i, list) for i in outputs):
    # Flatten the list if it's a list of lists
    outputs = [item for sublist in outputs for item in sublist]

# Decode the outputs
output_text = tokenizer.decode(outputs, skip_special_tokens=True)

print("SQL Query:")
print(output_text)



config.json:   0%|          | 0.00/813 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

SQL Query:
- count(*) FROM department WHERE age > 56


# schema-aware-denoising-bart-large-cnn-text2sql

Code link: https://huggingface.co/shahrukhx01/schema-aware-denoising-bart-large-cnn-text2sql

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('shahrukhx01/schema-aware-denoising-bart-large-cnn-text2sql')
tokenizer = BartTokenizer.from_pretrained('shahrukhx01/schema-aware-denoising-bart-large-cnn-text2sql')

## add NL query with table schema
# question = "What is terrence ross' nationality? </s> <col0> Player : text <col1> No. : text <col2> Nationality : text <col3> Position : text <col4> Years in Toronto : text <col5>  School/Club Team : text"
question = "How many heads of the departments are older than 56? </s> <col0> age : int"

inputs = tokenizer([question], max_length=1024, return_tensors='pt')

# Generate SQL
text_query_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=0, max_length=125, early_stopping=True)
prediction = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in text_query_ids][0]
print(prediction)


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


SELECT COUNT `  <col1>  ` FROM ` table ` WHERE `  <col0>  ` > ` 56 `


# schema-aware-distilbart-cnn-12-6-text2sql

Code: https://huggingface.co/shahrukhx01/schema-aware-distilbart-cnn-12-6-text2sql

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('shahrukhx01/schema-aware-distilbart-cnn-12-6-text2sql')
tokenizer = BartTokenizer.from_pretrained('shahrukhx01/schema-aware-distilbart-cnn-12-6-text2sql')

## add NL query with table schema
# question = "What is terrence ross' nationality? </s> <col0> Player : text <col1> No. : text <col2> Nationality : text <col3> Position : text <col4> Years in Toronto : text <col5>  School/Club Team : text"
question = "How many heads of the departments are older than 56? </s> <col0> age : int"

inputs = tokenizer([question], max_length=1024, return_tensors='pt')

# Generate SQL
text_query_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=0, max_length=125, early_stopping=True)
prediction = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in text_query_ids][0]
print(prediction)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


SELECT COUNT `  <col3>  ` FROM ` table ` WHERE `  <col0>  ` = ` 56 `
